In [ ]:
import numpy as np
import simphony.core as core
import simphony.errors as errors
import simphony.DeviceLibrary.ebeam as dev
import simphony.DeviceLibrary.sipann as sipann
import simphony.simulation as sim

import matplotlib.pyplot as plt
from scipy import special
import dill

In [ ]:
def bezier(x,b):
    def bernstein(n, j, t):
        return special.binom(n, j) * t ** j * (1 - t) ** (n - j)
    n = len(x)-1
    return {'g': x,
            'w': b,
            'f': lambda t: np.sum(np.array([(x[j])*bernstein(n,j,t/b) for j in range(len(x))]),axis=0),
            'df': lambda t: np.sum(np.array([n*(x[j]/2)*(bernstein(n-1,j-1,t/b)-bernstein(n-1,j,t/b)) for j in range(len(x))]),axis=0)/b,
            'd2f': lambda t: np.sum(np.array([n*(n-1)*(x[j]/2)*(bernstein(n-2,j-2,t/b)-2*bernstein(n-2,j-1,t/b)+bernstein(n-2,j,t/b)) for j in range(len(x))]),axis=0)/b**2}

## Get the Ports of Crossover

In [ ]:
dc = core.ComponentInstance(sipann.sipann_dc_crossover1550)
wg = core.ComponentInstance(dev.ebeam_wg_integral_1550)

c1 = [dc]
p1 = [1]
c2 = [wg]
p2 = [0]

con = zip(c1, p1, c2, p2)

nl = core.Netlist()
nl.load(con, formatter='ll')
simu = sim.Simulation(nl, num=30)

freq = simu.freq_array
sparams = simu.s_parameters()


plt.figure(figsize=(15,10))
plt.plot(freq, np.abs(sparams[:,0,1])**2, label='t')
plt.plot(freq, np.abs(sparams[:,0,2])**2, label='k')
c = 299792458
plt.axvline(c / 1550e-9, label='1550nm')
plt.legend(loc='upper right')
plt.show()

## Get the Through Port of a Racetrack Resonator using SiPANN and Simphony

In [ ]:
radius = 12
length = 4.5

bottom = core.ComponentInstance(sipann.sipann_dc_halfracetrack, extras=
                                {'radius': radius, 'length': length, 'gap': 0.3})

top    = core.ComponentInstance(sipann.sipann_dc_halfracetrack, extras=
                                {'radius': radius, 'length': length, 'gap': 0.3})

c1 = [bottom, bottom]
p1 = [1, 3]
c2 = [top, top]
p2 = [1, 3]

con = zip(c1, p1, c2, p2)

nl = core.Netlist()
nl.load(con, formatter='ll')
simu = sim.Simulation(nl)

freq = simu.freq_array
sparams = simu.s_parameters()


plt.figure(figsize=(15,10))
plt.plot(freq, np.abs(sparams[:,0,2]))
plt.show()

## Get the Ports of a 10/90 Optimized Directional Coupler

Note: This one may take a while to run.

In [ ]:
with open('dc.pkl', 'rb') as handle:
    dic = dill.load(handle)

pts = dic['g']
zmin = 0
zmax = dic['w']
dic = bezier(pts, zmax)
gap = dic['f']
dgap = dic['df']


dc = core.ComponentInstance(sipann.sipann_dc_arbitrarysym, extras=
                            {'gap': gap, 'dgap': dgap, 'zmin': zmin, 'zmax': zmax})
wg = core.ComponentInstance(dev.ebeam_wg_integral_1550)

c1 = [dc]
p1 = [1]
c2 = [wg]
p2 = [0]

con = zip(c1, p1, c2, p2)

nl = core.Netlist()
nl.load(con, formatter='ll')
simu = sim.Simulation(nl, num=30)

freq = simu.freq_array
sparams = simu.s_parameters()


plt.figure(figsize=(15,10))
plt.plot(freq, np.abs(sparams[:,0,1])**2, label='t')
plt.plot(freq, np.abs(sparams[:,0,2])**2, label='k')
plt.legend(loc='upper right')
plt.show()